In [1]:
import tensorflow as tf
import autokeras as ak

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[-1], 'GPU')

## Load MNIST dataset

In [7]:
from tensorflow.keras.datasets import cifar10, fashion_mnist, mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


## ResNetBlock

In [ ]:
import timeit

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ResNetBlock()(output_node) # version="v2", pretrained=False
output_node = ak.ClassificationHead()(output_node)

resnet_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
resnet_auto_model.fit(x_train, y_train, epochs=10, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

### Get the summarized results during the tuning process

In [ ]:
resnet_auto_model.tuner.results_summary()

### Display best model

In [ ]:
best_resnet_model = resnet_auto_model.export_model()
best_resnet_model.summary()

### Evaluate the best resnet model on the test data.

In [ ]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



## XceptionBlock

In [ ]:
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.XceptionBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

xception_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
xception_auto_model.fit(x_train, y_train, epochs=10, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

### Display the best xception model

In [ ]:
best_xception_model = xception_auto_model.export_model()
tf.keras.utils.plot_model(best_xception_model, show_shapes=True, expand_nested=True) # rankdir='LR'

In [ ]:
best_xception_model.summary()

### Evaluate the best xception model on the test data.

In [ ]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



## HyperBlock for image classification (ImageBlock)

In [21]:
import timeit
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.ImageBlock(
                # Normalize the dataset.
                normalize=True,
                # Do not do data augmentation.
                augment=False)(input_node)
output_node = ak.ClassificationHead(dropout=0.0)(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=10,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
auto_model.fit(x_train, y_train, epochs=3, batch_size=32) # , batch_size=64
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

Trial 10 Complete [00h 03m 38s]
val_loss: 0.09551089257001877

Best val_loss So Far: 0.06062331795692444
Total elapsed time: 01h 02m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 [==============================] - 78s 42ms/step - loss: 0.1326 - accuracy: 0.9614
Epoch 2/3
1875/1875 [==============================] - 78s 42ms/step - loss: 0.0576 - accuracy: 0.9836
Epoch 3/3
1875/1875 [==============================] - 78s 42ms/step - loss: 0.0425 - accuracy: 0.9877
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
Total time: 4008.61 seconds.


In [27]:
auto_model.tuner.results_summary(num_trials=1)

Results summary
Results in ./auto_model
Showing 1 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
image_block_1/block_type: xception
classification_head_1/spatial_reduction_1/reduction_type: global_avg
optimizer: adam
learning_rate: 0.001
image_block_1/xception_block_1/pretrained: False
image_block_1/xception_block_1/imagenet_size: False
Score: 0.06062331795692444


In [24]:
best_model = auto_model.export_model()
best_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 28, 28)       0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 28, 28, 1)    0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           expand_last_dim[0][0]            
_______________________________________________________________________________________

In [25]:

test_loss, test_acc = auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



Accuracy: 98.57%


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).